# Segmenting and Clustering Neighborhoods in New York City 
> We will explore neighborhoods in New York City and group them into clusters.

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [segmentation, k-means clustering]


In this blog post, we will explore neighborhoods in New York City using the Foursquare API. We will get the most common venue categories in each neighborhood, and then using the k-means clustering algorithm, group the neighborhoods into clusters.

In [122]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


---
### Explore the data

In [123]:
# download the data from a server
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded.')

Data downloaded.


In [0]:
# load the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [125]:
#collapse-output
# explore the data
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

All the relevant data is in the *features* key, which is basically a list of the neighborhoods.  
Let's define a new variable that includes this data.

In [0]:
neighborhoods_data = newyork_data['features']

Let's check the first item in the list.

In [127]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

---
### Transform the data into a pandas dataframe
We will essentially be transforming this data of nested Python dictionaries into a pandas dataframe.  
Let's start by creating an empty dataframe.

In [0]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [129]:
# empty dataframe
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Let's loop through the data and fill the dataframe one row at a time.

In [0]:
for data in neighborhoods_data:
  borough = neighborhood_name = data['properties']['borough']
  neighborhood_name = data['properties']['name']

  neighborhood_latlon = data['geometry']['coordinates']
  neighborhood_lat = neighborhood_latlon[1]
  neighborhood_lon = neighborhood_latlon[0]

  neighborhoods = neighborhoods.append({'Borough': borough, 
                                        'Neighborhood': neighborhood_name,
                                        'Latitude': neighborhood_lat,
                                        'Longitude': neighborhood_lon}, ignore_index=True)

In [131]:
# examine the dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [132]:
# check whether the dataframe has all 5 boroughs and 306 neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
    len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


---
### Visualize the data

We will use the geopy library to get the latitude and longitude values of New York City.  
In order to define an instance of the geocoder, we need to define a user_agent.

In [133]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of New York City are {}, {}.'.format(latitude, longitude))

The geographical coordinates of New York City are 40.7127281, -74.0060152.


Let's create a map of New York with the neighborhoods superimposed on top.

In [134]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

You can zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Let's simplify the above map, and segment and cluster only the neighborhoods in Manhattan. We will slice the original dataframe and create a new dataframe of the Manhattan data.

In [135]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographiical coordinates of Manhattan.

In [136]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Manhattan are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Manhattan are 40.7896239, -73.9598939.


Let's visualize the neighborhoods in Manhattan.

In [137]:
# create a map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map_manhattan)

map_manhattan

---
### Explore the neighborhoods in Manhattan
We will utilize the Foursquare API to explore the neighborhoods and segment them.

In [0]:
# define Foursquare Credentials
CLIENT_ID = 'B50RYBOWHJ3ZMRVEFTLHJCXGOYNCXGI13VDT5FMYTUQSUTQC' 
CLIENT_SECRET = 'ST2IN4ZGOQ3BXEWJ2HW1LKUML2BBGK1JX1QD2WKEPET31W4W'
VERSION = '20191219'

#### Explore the first neighborhood in our dataframe.

In [139]:
# get the neighborhood's name
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [140]:
# get the neighborhood's latitude and longitude values
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0,'Longitude'] #neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('The geographical coordinates of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

The geographical coordinates of Marble Hill are 40.87655077879964, -73.91065965862981.


Let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.  
First, create the GET request URL.

In [141]:
# limit the number of venues returned by the foursquare API
LIMIT = 100

# define radius
radius = 500

# Create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=B50RYBOWHJ3ZMRVEFTLHJCXGOYNCXGI13VDT5FMYTUQSUTQC&client_secret=ST2IN4ZGOQ3BXEWJ2HW1LKUML2BBGK1JX1QD2WKEPET31W4W&v=20191219&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

Send the GET request and examine the results.

In [142]:
#collapse-output
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e68ce37618f43001c2e7a8e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

All the information is in the *items* key.  
Let's borrow the `get_category_type` function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']

  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [144]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [145]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


#### Explore all neighborhoods in Manhattan

Let's create a function to repeat the same process for all the neighborhoods in Manhattan.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe.

In [147]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'], 
                                   longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the resulting dataframe.

In [148]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3313, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


Let's check how many venues were returned for each neighborhood.

In [149]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,95,95,95,95,95,95
Carnegie Hill,100,100,100,100,100,100
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


How many unique categories can be curated from all the returned venues?

In [150]:
print('There are {} unique categories'.format(len(manhattan_venues['Venue Category'].unique())))

There are 343 unique categories


---
### Analyze each neighborhood

In [151]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix='', prefix_sep='')

# add neighborhood column back to the dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbersh

In [152]:
# examine the new dataframe size
manhattan_onehot.shape

(3313, 344)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [153]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbersh

In [154]:
# confirm the new size
manhattan_grouped.shape

(40, 344)

Let's print the top 5 most common venues for each neighborhood.

In [155]:
#collapse-output
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
  print('----' + hood + '----')
  temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
  temp.columns = ['venue', 'freq']
  temp = temp.iloc[1:]
  temp['freq'] = temp['freq'].astype(float)
  temp = temp.round({'freq': 2})
  print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
  print('\n')

----Battery Park City----
           venue  freq
0           Park  0.07
1    Coffee Shop  0.06
2          Hotel  0.05
3  Shopping Mall  0.03
4  Women's Store  0.03


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.06
1           Pizza Place  0.06
2                  Café  0.05
3  Gym / Fitness Center  0.03
4        Cosmetics Shop  0.03


----Central Harlem----
                venue  freq
0  African Restaurant  0.07
1                 Bar  0.04
2  Seafood Restaurant  0.04
3   French Restaurant  0.04
4      Cosmetics Shop  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.05
2              Bakery  0.05
3               Hotel  0.03
4             Theater  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1           Cocktail Bar  0.05
2  Vietnamese Restaurant  0.04
3    American Restaurant  0.04
4                 Bakery  0.03


----Civic Center----
                  venue 

Let's put that into a pandas dataframe.

In [0]:
# funciton to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)

  return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [157]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Women's Store,Memorial Site,Shopping Mall,Gym,Wine Shop,Boat or Ferry,Food Court
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Bookstore,Cosmetics Shop,Grocery Store,Gym,Gym / Fitness Center,Bakery,Japanese Restaurant
2,Central Harlem,African Restaurant,Fried Chicken Joint,French Restaurant,American Restaurant,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Bar,Metro Station,Southern / Soul Food Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,Bakery,Wine Shop,American Restaurant,Ice Cream Shop,Theater,Hotel,Sushi Restaurant,Cupcake Shop
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Spa,Bakery,Salon / Barbershop,Bar


---
### Cluster the neighborhoods

Run the k-means algorithm to cluster the neighborhood into 5 clusters.

In [158]:
# set the number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 3, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [159]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Coffee Shop,Gym,Sandwich Place,Yoga Studio,Tennis Stadium,Supplement Shop,Donut Shop,Miscellaneous Shop,Steakhouse,Discount Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Spa,Bakery,Salon / Barbershop,Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Grocery Store,Bakery,Mobile Phone Shop,Pizza Place,Chinese Restaurant,Tapas Restaurant,Mexican Restaurant,Coffee Shop,Supplement Shop
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Spanish Restaurant,Bakery,Park,Chinese Restaurant,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School


Let's visualize the clusters.

In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

---
### Examine the clusters
You can examine and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can assign a name to each cluster.

**Cluster 1**

In [161]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Pizza Place,Deli / Bodega,Sushi Restaurant,Wine Shop,Mexican Restaurant,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Burger Joint,Gym / Fitness Center,Café,Gym,Sporting Goods Shop
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Café,Coffee Shop,Mediterranean Restaurant,Bakery,Indian Restaurant,Yoga Studio,Dessert Shop
13,Lincoln Square,Plaza,Café,Theater,Italian Restaurant,Concert Hall,Performing Arts Venue,Indie Movie Theater,American Restaurant,Gym / Fitness Center,Park
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,Seafood Restaurant,Indian Restaurant,French Restaurant,Cosmetics Shop,Gourmet Shop,Boutique
19,East Village,Bar,Wine Bar,Ice Cream Shop,Pizza Place,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Ramen Restaurant,Italian Restaurant,Japanese Restaurant
25,Manhattan Valley,Pizza Place,Indian Restaurant,Bar,Yoga Studio,Coffee Shop,Mexican Restaurant,Deli / Bodega,Thai Restaurant,Playground,Park
29,Financial District,Coffee Shop,American Restaurant,Bar,Gym,Pizza Place,Gym / Fitness Center,Hotel,Steakhouse,Food Truck,Event Space
30,Carnegie Hill,Pizza Place,Coffee Shop,Café,Bookstore,Cosmetics Shop,Grocery Store,Gym,Gym / Fitness Center,Bakery,Japanese Restaurant
33,Midtown South,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Dessert Shop,Gym / Fitness Center,Coffee Shop,Cosmetics Shop,American Restaurant,Fried Chicken Joint


**Cluster 2**

In [162]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop,Spa,Bakery,Salon / Barbershop,Bar
6,Central Harlem,African Restaurant,Fried Chicken Joint,French Restaurant,American Restaurant,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Bar,Metro Station,Southern / Soul Food Restaurant
8,Upper East Side,Art Gallery,Italian Restaurant,Exhibit,Coffee Shop,Bakery,Juice Bar,Gym / Fitness Center,French Restaurant,Hotel,Pizza Place
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Coffee Shop,Sandwich Place,Wine Shop,Spa,Hotel,Lounge
15,Midtown,Hotel,Coffee Shop,Clothing Store,Bakery,Steakhouse,French Restaurant,Café,Theater,Sporting Goods Shop,Cocktail Bar
16,Murray Hill,Coffee Shop,Sandwich Place,Japanese Restaurant,American Restaurant,Hotel,Gym,Italian Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Bagel Shop
17,Chelsea,Coffee Shop,Italian Restaurant,Bakery,Wine Shop,American Restaurant,Ice Cream Shop,Theater,Hotel,Sushi Restaurant,Cupcake Shop
20,Lower East Side,Pizza Place,Coffee Shop,Café,Art Gallery,Japanese Restaurant,Ramen Restaurant,Chinese Restaurant,Cocktail Bar,Bakery,Filipino Restaurant
21,Tribeca,Park,American Restaurant,Italian Restaurant,Spa,Café,Wine Shop,Wine Bar,Boutique,Coffee Shop,Greek Restaurant
22,Little Italy,Café,Bakery,Bubble Tea Shop,Sandwich Place,Salon / Barbershop,Italian Restaurant,Mediterranean Restaurant,Cocktail Bar,Yoga Studio,Women's Store


**Cluster 3**

In [163]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Baseball Field,Pet Service,Gas Station,Boat or Ferry,Farmers Market,Gym / Fitness Center,Cocktail Bar,Harbor / Marina


**Cluster 4**

In [164]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Grocery Store,Bakery,Mobile Phone Shop,Pizza Place,Chinese Restaurant,Tapas Restaurant,Mexican Restaurant,Coffee Shop,Supplement Shop
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Restaurant,Café,Spanish Restaurant,Bakery,Park,Chinese Restaurant,Frozen Yogurt Shop
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Yoga Studio,Sushi Restaurant,Caribbean Restaurant,Chinese Restaurant,School
5,Manhattanville,Coffee Shop,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Park,Indian Restaurant,Supermarket,Boutique,Spanish Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Deli / Bodega,French Restaurant,Spa,Liquor Store,Taco Place,Gas Station
11,Roosevelt Island,Park,Coffee Shop,Sandwich Place,Kosher Restaurant,Gym / Fitness Center,Gym,Greek Restaurant,Dry Cleaner,Outdoors & Recreation,Playground
26,Morningside Heights,Bookstore,Park,American Restaurant,Coffee Shop,Burger Joint,Sandwich Place,Deli / Bodega,Café,Seafood Restaurant,Salad Place
36,Tudor City,Park,Mexican Restaurant,Café,Deli / Bodega,Diner,Pizza Place,Coffee Shop,Greek Restaurant,Dog Run,Thai Restaurant


**Cluster 5**

In [165]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Gym,Sandwich Place,Yoga Studio,Tennis Stadium,Supplement Shop,Donut Shop,Miscellaneous Shop,Steakhouse,Discount Store


Cluster 2 is the biggest. Cluster 3 and 5 have only one neighborhood each.